In [31]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns',None)

In [32]:
df_home=pd.read_csv('/Users/aurelientarroux/Desktop/ML/Data_Challenge_QRT/Data/Train_Data/train_home_team_statistics_df.csv')
df_away=pd.read_csv('/Users/aurelientarroux/Desktop/ML/Data_Challenge_QRT/Data/Train_Data/train_home_team_statistics_df.csv')

df_home.columns='home_'+df_home.columns
df_away.columns='away_'+df_away.columns

df_result=pd.read_csv('/Users/aurelientarroux/Desktop/ML/Data_Challenge_QRT/Data/Y_train_1rknArQ.csv')

base_match=pd.merge(df_home,df_away,left_on="home_ID",right_on='away_ID',how='inner')
base=pd.merge(base_match,df_result,left_on="home_ID",right_on='ID',how='inner')
base=base.drop(['home_ID','home_LEAGUE','home_TEAM_NAME','away_ID','away_LEAGUE','away_TEAM_NAME'],axis=1)

def resultat(row):
    if row['HOME_WINS']==1:
        return 1
    elif row['AWAY_WINS']==1:
        return 0
    else:
        return 2

base['Resultat']=base.apply(resultat, axis=1)
base=base.drop(['HOME_WINS','AWAY_WINS','DRAW'],axis=1)

base.dropna(inplace=True)
base=base[base['Resultat'].isin([0,1])]

X = base.drop(['Resultat','ID'],axis=1)
y = base['Resultat']
X=X.iloc[:,80:200]

def centre_reduire(x,mean,std):
    return (x-mean)/std

for colonne in X.columns:
    mean=X[colonne].mean()
    std=X[colonne].std()
    X[colonne]=X[colonne].apply(lambda x: centre_reduire(x,mean,std))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X.sum(axis=0)


home_TEAM_YELLOWCARDS_5_last_match_sum              -6.821210e-13
home_TEAM_REDCARDS_5_last_match_sum                 -3.907985e-14
home_TEAM_OFFSIDES_5_last_match_sum                 -7.460699e-13
home_TEAM_ATTACKS_5_last_match_sum                   3.694822e-13
home_TEAM_PENALTIES_5_last_match_sum                 1.794120e-13
                                                         ...     
away_TEAM_SUCCESSFUL_PASSES_PERCENTAGE_season_std   -2.842171e-13
away_TEAM_SAVES_season_std                          -7.958079e-13
away_TEAM_CORNERS_season_std                        -6.679102e-13
away_TEAM_BALL_POSSESSION_season_std                -1.136868e-13
away_TEAM_FOULS_season_std                           1.044498e-12
Length: 120, dtype: float64

In [33]:
def fonction(param):
    compte_total=0

    for i in range(len(X_train.index)): #pour chaque individus
        
        compte_exp_individus=0
        
        for k in range(len(X_train.columns)):
            compte_exp_individus+= X_train.iloc[i,k]*param[k]
        
        compte_total += ( ( y_train.iloc[i] * np.log( 1 / ( 1 + np.exp( -compte_exp_individus ) ) ) ) + ( ( 1 - y_train.iloc[i] ) * np.log( 1 - ( 1 / ( 1 + np.exp( - compte_exp_individus ) ) ) ) ) ) 
    print(-compte_total)
    return -compte_total


In [34]:

from scipy.optimize import minimize
import numpy as np
initial_guess = np.zeros(len(X_train.columns))
result = minimize(fonction, initial_guess, method='L-BFGS-B')

print("Paramètres optimisés :", result.x)
print("Valeur de la fonction objective :", result.fun)
print("Convergence :", result.success)

3465.0427556193986
3465.042756281385
3465.0427561809324
3465.042753651492
3465.0427517705366
3465.0427536731154
3465.0427554626663
3465.042754406713
3465.042751221737
3465.0427567302163
3465.0427516317145
3465.042751319746
3465.0427559353566
3465.0427593872014
3465.0427513120567
3465.0427508417793
3465.042753479242
3465.042751232937
3465.042754106284
3465.04275103352
3465.0427510023533
3465.0427514681764
3465.0427580294627
3465.042752065247
3465.042751228325
3465.0427571068253
3465.042756299086
3465.042756180924
3465.0427537380697
3465.0427517518856
3465.0427536785214
3465.0427555264428
3465.042754658261
3465.0427512389415
3465.0427565447726
3465.042751647518
3465.042751331323
3465.0427559353566
3465.0427593823856
3465.042754156582
3465.0427545473485
3465.042754647038
3465.0427538966046
3465.0427545997727
3465.042754237811
3465.0427541130357
3465.042757577559
3465.042756462158
3465.0427543746723
3465.0427559505224
3465.0427562956484
3465.0427558990796
3465.0427559490777
3465.0427545515

KeyboardInterrupt: 

In [ ]:

y_pred=pd.DataFrame(index=y_test.index)

for index in X_test.index:        
    
    compte_exp_individus=0
    for colonne in range(len(X_train.columns)):
        compte_exp_individus += X_test.iloc[:,colonne][index]*optimal_params[colonne]
    proba=1/(1+np.exp(-compte_exp_individus))
    
    if proba>0.5:
        y_pred.loc[index,'Resultat']=int(1)
    else:
        y_pred.loc[index,'Resultat']=int(0)
df_total=pd.DataFrame(y_pred==pd.DataFrame(y_test))
print(len(df_total[df_total['Resultat']==True])/len(df_total))


0.5672
